## Example of 3D Graph

In [1]:
import os
import torch
from torch_geometric.data import DataLoader
import sys
sys.path.insert(0,'..')
sys.path.insert(0,'../..')
from dig.threedgraph.dataset import QM9_3D
from dig.threedgraph.method import SchNet
from dig.threedgraph.method import DimeNetPP
from dig.threedgraph.method import SphereNet
from dig.threedgraph.method import run
from dig.threedgraph.evaluation import threedEvaluator

In [2]:
device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device("cpu")

### Prepare Dataset

In [3]:
dataset = QM9_3D(root='dataset/')
target = 'U0'
dataset.data.y = dataset.data[target]

split_idx = dataset.get_idx_split(len(dataset.data.y), train_size=110000, valid_size=10000, seed=42)

train_dataset, valid_dataset, test_dataset = dataset[split_idx['train']], dataset[split_idx['valid']], dataset[split_idx['test']]
print('train, validaion, test:', len(train_dataset), len(valid_dataset), len(test_dataset))

train, validaion, test: 110000 10000 10831


### Training

In [4]:
model = SchNet()
loss_func = torch.nn.L1Loss()
evaluation = threedEvaluator()

In [5]:
run3d = run()
run3d.run(train_dataset, valid_dataset, test_dataset, model, loss_func, evaluation, epochs=10, batch_size=128, lr=0.001, lr_decay_factor=0.5, lr_decay_step_size=5)


=====Epoch 1

Training...
100%|██████████| 860/860 [01:25<00:00, 10.01it/s]

Evaluating...

100%|██████████| 79/79 [00:05<00:00, 13.58it/s]

Testing...

100%|██████████| 85/85 [00:05<00:00, 14.62it/s]
{'Train': 0.9855780021915602, 'Validation': 0.22591181099414825, 'Test': 0.22721393406391144}

=====Epoch 2

Training...

100%|██████████| 860/860 [00:51<00:00, 16.73it/s]

Evaluating...

100%|██████████| 79/79 [00:02<00:00, 32.97it/s]

Testing...

100%|██████████| 85/85 [00:02<00:00, 31.71it/s]
{'Train': 0.26047287720233897, 'Validation': 0.2600099444389343, 'Test': 0.25914502143859863}

=====Epoch 3

Training...

100%|██████████| 860/860 [00:51<00:00, 16.76it/s]

Evaluating...

100%|██████████| 79/79 [00:02<00:00, 33.81it/s]

Testing...

100%|██████████| 85/85 [00:02<00:00, 30.55it/s]
{'Train': 0.1719289511788723, 'Validation': 0.14316318929195404, 'Test': 0.14212408661842346}

=====Epoch 4

Training...

100%|██████████| 860/860 [00:51<00:00, 16.84it/s]

Evaluating...

100%|██████████|